In [2]:
import numpy as np
import joblib

# Load the model
model = joblib.load('random_forest_genre_classifier.pkl')

# Define the list of classes in the same order as your model was trained
classes = model.classes_

def predict_genre(input_vector):
    """
    Predict music genre from a 1x16 vector.
    
    Args:
        input_vector (list or np.array): Shape (16,)
    
    Returns:
        str: Predicted genre
    """
    input_vector = np.array(input_vector).reshape(1, -1)
    prediction = model.predict(input_vector)
    return prediction[0]


In [3]:
# vector = [0.905594406, 0.374125874, 0.461538462, 0.699300699, 0, 1, 0, 0.108391608, 0, 0.958041958, 0, 0.685314685, 0.20979021, 0.58041958, 0.444055944, 0.531468531, 0.102102102, 0.318318318, 0.378378378, 0.708708709, 0, 0.813813814, 0.066066066, 0.177177177, 0.564564565, 0.381381381, 0.216216216, 0.927927928, 1, 0.54954955, 0.408408408, 0.840840841, 0, 0.090909091, 0.451048951, 0.755244755, 0, 1, 0, 0, 0.13986014, 0.629370629, 0.601398601, 0.888111888, 0.293706294, 0.444055944, 0.031468531, 0.552447552, 0.104986877, 0.291338583, 0.322834646, 0.56167979, 0.330708661, 0.603674541, 0.199475066, 0.333333333, 0.257217848, 0.94488189, 0.503937008, 0.627296588, 0.619422572, 1, 0.419947507, 0.908136483]
# predict_genre(vector)

import pandas as pd

source = pd.read_excel("testing_model_cnn.xlsx")
# from this source.iloc[0] i want to get all the features from 0 to 15
# and put them in a list

cols = len(source.columns)

source.shape[0]
count = 0
for i in range(source.shape[0]-1):
	vector = source.iloc[i, 3:cols].tolist()
	
	print(f"real: {source.iloc[i]['class']}")
	# print(vector)
	# print(predict_genre(vector))
	predicted = predict_genre(vector)
	print(f"predicted: {predicted}")
	if predicted in source.iloc[i]['class']:
		count += 1
		print("correct")
	print("="*50)

# vector = source.iloc[1468, 3:cols].tolist()

# predict_genre(vector)
print(f"{count}/{source.shape[0]} correct")

real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: soul/groove10
predicted: funk
real: funk/groove1
predicted: funk
correct
real: funk/groove1
predicted: funk
correct
real: funk/groove1
predicted: funk
correct
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove1
predicted: dance
real: funk/groove2
predicted: funk
correct
real: funk/groove2
predicted: funk
correct
real: funk/groove2
predicted:

In [9]:
import numpy as np
import pandas as pd

# Configuración
CSV_PATH = "/Users/danielmartinezvillegas/Developer/master-ds/✨TDG/tesis_work_space/combined_drum_database.csv"
OUTPUT_PATH = "predicted_rhythms.xlsx"

def export_test_set_with_predictions():
    """
    Versión mínima que no depende de ningún modelo.
    Simplemente carga el CSV, simula predicciones y exporta.
    """
    print(f"Cargando CSV desde {CSV_PATH}...")
    
    # Cargar datos
    try:
        df = pd.read_csv(CSV_PATH)
        print(f"CSV cargado con éxito. {len(df)} filas encontradas.")
    except Exception as e:
        print(f"Error al cargar el CSV: {e}")
        return None
    
    # Filtrar solo ejemplos de test
    test_df = df[df['split'] == 'test'].copy()
    print(f"Conjunto de prueba: {len(test_df)} ejemplos")
    
    # Simular predicciones para cada ejemplo
    print("Generando predicciones simuladas...")
    
    for i in range(len(test_df)):
        # Obtener los valores reales (ground truth)
        y_true = []
        for j in range(16):
            y_key = f'y_{j}'
            if y_key in test_df.columns:
                y_true.append(test_df.iloc[i][y_key])
            else:
                # Si no hay columnas y_j, usamos las low_j como aproximación
                y_true.append(test_df.iloc[i][f'low_{j}'])
        
        # Simular predicciones (en un caso real, estas vendrían de tu modelo)
        for j in range(16):
            # Simulamos predicciones cercanas a los valores reales + ruido
            pred = y_true[j] * 0.8 + 0.1 + np.random.normal(0, 0.05)
            # Aseguramos que estén en el rango [0, 1]
            pred = max(0.0, min(1.0, pred))
            # Guardar en el dataframe
            test_df.loc[test_df.index[i], f'step_{j}'] = pred
        
        # Calcular el MSE para este ejemplo
        mse = np.mean([(y_true[j] - test_df.iloc[i][f'step_{j}'])**2 for j in range(16)])
        test_df.loc[test_df.index[i], 'mse'] = mse
    
    # Ordenar por MSE (mejores predicciones primero)
    test_df = test_df.sort_values('mse')
    
    # Guardar a CSV
    test_df.to_excel(OUTPUT_PATH, index=False)
    print(f"Predicciones guardadas en {OUTPUT_PATH}")
    
    # También guardar los 100 mejores ejemplos
    best_df = test_df.head(100)
    best_df.to_csv("best_predictions.csv", index=False)
    print(f"Los 100 mejores ejemplos guardados en best_predictions.csv")
    
    # Mostrar estadísticas
    print("\nEstadísticas:")
    print(f"MSE promedio: {test_df['mse'].mean():.6f}")
    print(f"MSE mejor ejemplo: {test_df['mse'].min():.6f}")
    print(f"MSE peor ejemplo: {test_df['mse'].max():.6f}")
    
    return test_df

if __name__ == "__main__":
    # Ejecutar la función de exportación
    export_test_set_with_predictions()

Cargando CSV desde /Users/danielmartinezvillegas/Developer/master-ds/✨TDG/tesis_work_space/combined_drum_database.csv...
CSV cargado con éxito. 17655 filas encontradas.
Conjunto de prueba: 1837 ejemplos
Generando predicciones simuladas...
Predicciones guardadas en predicted_rhythms.xlsx
Los 100 mejores ejemplos guardados en best_predictions.csv

Estadísticas:
MSE promedio: 0.007619
MSE mejor ejemplo: 0.001753
MSE peor ejemplo: 0.018485


In [41]:
a = 'funk/asdasd'

'funk' in a

True

In [4]:
(3103/15541)*100


19.96654011968342